<a href="https://colab.research.google.com/github/klnsuman/MyProjects/blob/main/TextClassification_PreTrained_Word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string
print(gensim.__version__)

4.3.1


In [6]:
np.random.seed(42)

In [7]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [8]:
wv = api.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded


In [19]:
wv_news = api.load('word2vec-google-news-300')

[=================================================-] 99.9% 1661.5/1662.8MB downloaded


In [24]:
wv_news.save('/content/drive/MyDrive/Data/wv_news_vectors.kv')

In [9]:
type(wv)

gensim.models.keyedvectors.KeyedVectors

In [20]:
wv['apple'],wv_news['apple']

(array([ 5.1418e-01, -1.9196e-01, -7.9963e-01,  9.1339e-01, -5.7061e-01,
         7.1746e-01, -5.8979e-01, -5.1530e-01,  1.3453e+00, -2.0792e-01,
         4.4555e-01,  2.4981e-01, -3.2020e+00,  3.5736e-01, -2.3593e-02,
        -3.1158e-02,  3.7782e-01,  6.4447e-01,  1.2056e+00, -1.0553e-01,
        -8.9697e-01, -7.9754e-01,  2.0239e-01, -1.0664e+00, -4.7167e-01,
        -2.5953e-01,  1.1328e-01, -3.8142e-01,  2.0366e-03, -5.0261e-01,
        -2.5197e-01, -8.4033e-02, -1.0015e+00, -8.6835e-01,  6.6201e-01,
         2.6496e-01,  2.2570e-01, -3.3847e-01, -3.4739e-01,  6.2199e-01,
        -1.3491e+00, -1.7497e-01, -1.1490e+00,  1.5144e-01,  1.1255e+00,
        -6.2668e-01, -3.4274e-01, -3.4938e-01, -1.0806e+00,  6.0174e-01],
       dtype=float32),
 array([-0.06445312, -0.16015625, -0.01208496,  0.13476562, -0.22949219,
         0.16210938,  0.3046875 , -0.1796875 , -0.12109375,  0.25390625,
        -0.01428223, -0.06396484, -0.08056641, -0.05688477, -0.19628906,
         0.2890625 , -0.051

In [21]:
len(wv['apple']),len(wv_news['apple'])

(50, 300)

In [23]:
print(wv.similarity("apple", "mango"),wv_news.similarity("apple", "mango"))
print(wv.similarity("apple", "car"),wv_news.similarity("apple", "car"))

0.5289788 0.57518554
0.51229525 0.12830707


In [14]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.48
'car'	'bicycle'	0.62
'car'	'airplane'	0.67
'car'	'cereal'	0.38
'car'	'communism'	0.01


In [16]:
data = pd.read_csv("/content/drive/MyDrive/Data/toxic_comments.csv",error_bad_lines=False, engine="python")
data.head()

<ipython-input-16-961a6967bfae>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/drive/MyDrive/Data/toxic_comments.csv",error_bad_lines=False, engine="python")


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [25]:
wv = wv_news

In [26]:
print(wv,wv.vector_size)

KeyedVectors<vector_size=300, 3000000 keys> 300


In [29]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [30]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [31]:
sent_vec("I am happy")

array([-0.08447266,  0.10973685, -0.00453404,  0.18362863, -0.05944824,
       -0.04013497, -0.10041155,  0.03710938, -0.02503313,  0.04729353,
       -0.07543836, -0.16434152, -0.1116333 ,  0.02915737, -0.12290737,
        0.10421317, -0.00425502,  0.27535575, -0.01681083, -0.08075387,
       -0.26747349, -0.07281712,  0.08138602, -0.02498954, -0.05358887,
       -0.07969884, -0.30385045,  0.10531616, -0.03756278, -0.04029192,
        0.06455776, -0.02200753, -0.11361694, -0.12702288, -0.19824219,
        0.06921387, -0.14557757,  0.18457031, -0.05542864,  0.10093471,
        0.10728237, -0.07920619, -0.01236398,  0.11485073,  0.09437779,
        0.02020509, -0.08954729, -0.12911551, -0.0363072 ,  0.05212402,
       -0.12200056,  0.23856027, -0.04056222,  0.12512207,  0.05569894,
        0.1237793 , -0.08370536, -0.05489676, -0.0291748 , -0.18401228,
       -0.14592634, -0.04910714, -0.15844727, -0.03636387,  0.00628662,
       -0.30032785, -0.11404855,  0.08530971,  0.01077706,  0.05

In [32]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'say', 'third', 'over', 'why', 'perhaps', 'even', 'except', 'else', 'us', 'such', 'regarding', 'take', 'had', 'front', 'against', 'whom', 'none', 'go', 'might', 'after', 'itself', 'own', 'someone', 'did', 'seeming', 'everywhere', 'when', 'is', 'an', 'ca', 'hereupon', 'these', 'along', 'its', 'she', 'most', 'using', 'above', 'fifteen', 'mine', 'everyone', 'namely', 'to', 'either', 'themselves', 'four', 'my', 'still', 'here', 'formerly', 'with', 'others', 'have', 'some', 'beyond', 'already', 'off', 'because', 'where', 'fifty', 'more', 'whence', 'both', 'amongst', 'sometime', 'done', 'almost', 'quite', 'afterwards', 'cannot', 'nevertheless', 'very', 'two', 'become', 'noone', 'full', 'rather', 'however', 'used', 'thru', 'being', 'down', 'seem', 'no', 'or', 'those', 'the', 'anyway', 'of', 'amount', 'from', '‘d', 'least', 'himself', 'hereafter', 'anyhow', '’d', 'by', 'also', 'ever', 'her', 'throughout', 'latterly', 'wherein', 'yet', 'this', 'up', 'please', 'mostly', 'whereafter', 'doing', '

In [33]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [34]:
data['tokens'] = data['comment_text'].apply(spacy_tokenizer)

In [35]:
data['tokens'].head()

0    [explanation, edit, username, hardcore, metall...
1    [d'aww, match, background, colour, seemingly, ...
2    [hey, man, try, edit, war, guy, constantly, re...
3    [real, suggestion, improvement, wonder, sectio...
4                  [sir, hero, chance, remember, page]
Name: tokens, dtype: object

In [36]:
data['vec'] = data['tokens'].apply(sent_vec)

In [37]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,tokens,vec
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edit, username, hardcore, metall...","[-0.005419776553199405, 0.046755836123511904, ..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[d'aww, match, background, colour, seemingly, ...","[-0.03402879503038195, -0.042989095052083336, ..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, try, edit, war, guy, constantly, re...","[0.03130035400390625, 0.043255615234375, -0.00..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[real, suggestion, improvement, wonder, sectio...","[0.008553277878534226, 0.006958734421502976, 0..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]","[0.14371744791666666, 0.051432291666666664, 0...."


In [38]:
X = data['vec'].to_list()
y = data['toxic'].to_list()

In [41]:
X[0]

array([-0.00541978,  0.04675584,  0.00133696,  0.0850801 , -0.06447638,
       -0.00180263, -0.00253596, -0.11783854,  0.12369792,  0.02642146,
       -0.02159773, -0.03835333, -0.02583386,  0.02868071, -0.09855725,
        0.0882859 ,  0.01708984,  0.05018398, -0.01936704, -0.13048444,
        0.074941  ,  0.06057485,  0.08355422,  0.03940982, -0.03506034,
        0.03308033, -0.05092076,  0.09277489,  0.04749698, -0.10054706,
       -0.04815383,  0.01269531, -0.09663028, -0.00693476, -0.02238973,
       -0.0322905 ,  0.08835856,  0.03654298,  0.06050037,  0.1015625 ,
        0.04052734, -0.04001726,  0.16807138, -0.01393345, -0.0494363 ,
       -0.10120501, -0.00805664, -0.02898362, -0.09638904,  0.03300985,
       -0.10766166, -0.01426442, -0.03711337, -0.08377511,  0.00310989,
       -0.01328096, -0.05810547, -0.06438337, -0.023888  , -0.09505208,
       -0.02969215, -0.00195022, -0.02789452, -0.03556315, -0.06668381,
       -0.02664621, -0.0712818 ,  0.02684675, -0.03303019,  0.12

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

In [42]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [43]:
classifier.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [44]:
from sklearn import metrics
predicted = classifier.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.950430831897227
Logistic Regression Precision: 0.8414239482200647
Logistic Regression Recall: 0.5949656750572082
